In [1]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 4.7 MB/s 
     |████████████████████████████████| 384 kB 35.0 MB/s 
     |████████████████████████████████| 140 kB 71.8 MB/s 
     |████████████████████████████████| 58 kB 6.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


In [2]:
!pip install -U selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from tqdm.auto import tqdm

In [5]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 20 not upgraded.
Need to get 95.6 MB of archives.
After this operation, 321 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 108.0.5359.71-0ubuntu0.18.04.5 [1,159 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 108.0.5359.71-0ubuntu0.18

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [7]:
columns = ["Name", "Feild of work", "Author's Pub", "Adj.#", "Citations", "h-index", "i10-index", "HomePage", "ORCID", "Institution", "Count for ins", "Faculty"]

In [36]:
url = "https://csrankings.org/#/index?graph&chi&robotics&bio&visualization&ecom&europe"

In [37]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [38]:
driver.get(url)

In [39]:
driver.current_url

'https://csrankings.org/#/index?graph&chi&robotics&bio&visualization&ecom&europe'

In [40]:
rows = 1+len(driver.find_elements(By.XPATH,
    '//table[@id="ranking"]/tbody/tr'))

rows

#/html/body/div[5]/form/div/div[2]/div[2]/div/div/table/tbody/tr

91

In [41]:
fac_pth = '//table[@id="ranking"]/tbody/tr[3]/td/div/div/table/tbody/tr[1]/td[2]/small/a[1]'

x = driver.find_element(By.XPATH, fac_pth).get_attribute("text")
x

'Christian Holz 0001'

In [42]:
fac_list = []
cc = 0
for r in tqdm(range(1, rows, 3)):
  pth = '//table[@id="ranking"]/tbody/tr['+str(r)+']/td[2]/span[2]'
  pth1 = '//table[@id="ranking"]/tbody/tr['+str(r)+']/td[3]'
  pth2 = '//table[@id="ranking"]/tbody/tr['+str(r)+']/td[4]'

  Fac_rows = 1+len(driver.find_elements(By.XPATH,
    '//table[@id="ranking"]/tbody/tr['+str(r+2)+']/td/div/div/table/tbody/tr'))
  
  
  for fr in range(1, Fac_rows, 2):
    all_fac = {}
    
    #Faculty First Page
    fac_pth = '//table[@id="ranking"]/tbody/tr['+str(r+2)+']/td/div/div/table/tbody/tr['+str(fr)+']/td[2]/small/a[1]'
    fac_pubs = '//table[@id="ranking"]/tbody/tr['+str(r+2)+']/td/div/div/table/tbody/tr['+str(fr)+']/td[3]/small/a'
    fac_adj = '//table[@id="ranking"]/tbody/tr['+str(r+2)+']/td/div/div/table/tbody/tr['+str(fr)+']/td[4]'

    x = driver.find_element(By.XPATH, fac_pth).get_attribute("text")
    y = driver.find_element(By.XPATH, fac_pubs).get_attribute("text")
    z = driver.find_element(By.XPATH, fac_adj).get_attribute('innerText')

    #Faculty second Page
    area_rows = 1+len(driver.find_elements(By.XPATH, '//table[@id="ranking"]/tbody/tr['+str(r+2)+']/td/div/div/table/tbody/tr['+str(fr)+']/td[2]/small/span[1]/span'))

    area = ''

    for ar in range(1, area_rows):
      fac_area = '//table[@id="ranking"]/tbody/tr['+str(r+2)+']/td/div/div/table/tbody/tr['+str(fr)+']/td[2]/small/span[1]/span['+str(ar)+']'
      a = driver.find_element(By.XPATH, fac_area).get_attribute('innerText').capitalize()
      if(area==''):
        area=area+a
      else:
        area=area+", "+a

    #Faculty Googler Scholer page
    google_url = '//table[@id="ranking"]/tbody/tr['+str(r+2)+']/td/div/div/table/tbody/tr['+str(fr)+']/td[2]/small/a[3]'
    nurl = driver.find_element(By.XPATH, google_url).get_attribute("href")
    ndriver = webdriver.Chrome('chromedriver',options=chrome_options)
    ndriver.get(nurl)

    try:
      Tcitation = ndriver.find_element(By.XPATH, '//table[@id="gsc_rsb_st"]/tbody/tr[1]/td[2]').get_attribute("innerText")
      hindex = ndriver.find_element(By.XPATH, '//table[@id="gsc_rsb_st"]/tbody/tr[2]/td[2]').get_attribute("innerText")
      i10_index = ndriver.find_element(By.XPATH, '//table[@id="gsc_rsb_st"]/tbody/tr[3]/td[2]').get_attribute("innerText")
      homepage = ndriver.find_element(By.XPATH, '//div[@id="gsc_prf_ivh"]/a').get_attribute("href")
    except:
      Tcitation = ''
      hindex = ''
      i10_index = ''
      homepage = ''
      

    #DBPL Page
    dbplurl = driver.find_element(By.XPATH, fac_pubs).get_attribute("href")
    dbpldriver = webdriver.Chrome('chromedriver',options=chrome_options)
    try:
      dbpldriver.get(dbplurl)
    except:
      continue

    #data binding
    all_fac["Name"] = x
    all_fac["Author's Pub"] = y
    all_fac["Adj.#"] = z
    all_fac["Feild of work"] = area
    all_fac["Citations"] = Tcitation
    all_fac["h-index"] = hindex
    all_fac["i10-index"] = i10_index
    all_fac["HomePage"] = homepage

    try:
      dbpl = dbpldriver.find_element(By.XPATH, '//header[@id="headline"]/nav/ul/li/div/a').get_attribute("href")
      remove = "https://orcid.org/"
      all_fac["ORCID"] = dbpl.replace(remove, '')
    except:
      print("No ORCID")

    
    all_fac["Institution"] = driver.find_element(By.XPATH, pth).text
    all_fac["Count for ins"] = driver.find_element(By.XPATH, pth1).text
    all_fac["Faculty"] = driver.find_element(By.XPATH, pth2).text

    fac_list.append(all_fac)
    cc+=1
    print(cc)
    
fac_list

  0%|          | 0/30 [00:00<?, ?it/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


[{'Name': 'Christian Holz 0001',
  "Author's Pub": '51',
  'Adj.#': '11.9',
  'Feild of work': 'Hci',
  'Citations': '5067',
  'h-index': '39',
  'i10-index': '69',
  'HomePage': 'https://www.christianholz.net/',
  'ORCID': '0000-0001-9655-9519',
  'Institution': 'ETH Zurich',
  'Count for ins': '7.7',
  'Faculty': '19'},
 {'Name': 'Stelian Coros',
  "Author's Pub": '50',
  'Adj.#': '12.3',
  'Feild of work': 'Graphics, Robotics',
  'Citations': '',
  'h-index': '',
  'i10-index': '',
  'HomePage': '',
  'ORCID': '0000-0001-6604-4784',
  'Institution': 'ETH Zurich',
  'Count for ins': '7.7',
  'Faculty': '19'},
 {'Name': 'Markus H. Gross',
  "Author's Pub": '46',
  'Adj.#': '9.7',
  'Feild of work': '',
  'Citations': '45385',
  'h-index': '107',
  'i10-index': '415',
  'HomePage': 'http://graphics.ethz.ch/~grossm',
  'ORCID': '0000-0002-6838-9775',
  'Institution': 'ETH Zurich',
  'Count for ins': '7.7',
  'Faculty': '19'},
 {'Name': 'Olga Sorkine-Hornung',
  "Author's Pub": '40',
  '

In [43]:
df = pd.DataFrame(fac_list, columns = columns)
df.to_csv('Faculty_eu.csv', index=False)

In [44]:
from google.colab import files
files.download('Faculty_eu.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
all_uni = driver.find_element(By.XPATH, '//table[@id="ranking"]/tbody/tr[1]/td[2]/span[2]')

In [ ]:
all_uni.text

In [ ]:
# for row in all_uni[1:]:
#   print(row.find_elements(By.TAG_NAME, "td")[2].text)
#   print(row.find_elements(By.TAG_NAME, "td")[3].text)

In [ ]:
dbplurl = "https://dblp.org/pid/02/9524.html"
dbpldriver = webdriver.Chrome('chromedriver',options=chrome_options)
dbpldriver.get(dbplurl)

In [ ]:
dbpl = dbpldriver.find_element(By.XPATH, '//header[@id="headline"]/nav/ul/li/div/a').get_attribute("href")
print(dbpl)